# Install Dependecies

In [ ]:
!pip install transformers streamlit Pillow matplotlib torch timm einops flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 89.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... 

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 0.939s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.5 MB/s eta 0:00:00


# Code

In [ ]:
%%writefile app.py

import streamlit as st
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
import torch
import numpy as np

# Initialize Florence-2-large model and processor
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# Function to resize and preprocess image
def preprocess_image(image_path, max_size=(800, 800)):
    image = Image.open(image_path).convert('RGB')
    if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
        image.thumbnail(max_size, Image.LANCZOS)

    # Convert image to numpy array
    image_np = np.array(image)

    # Ensure the image is in the format [height, width, channels]
    if image_np.ndim == 2:  # Grayscale image
        image_np = np.expand_dims(image_np, axis=-1)
    elif image_np.shape[0] == 3:  # Image in [channels, height, width] format
        image_np = np.transpose(image_np, (1, 2, 0))

    return image_np, image.size

# Function to run Florence-2-large model
def run_florence_model(image_np, image_size, task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input

    inputs = processor(text=prompt, images=image_np, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"].cuda(),
            pixel_values=inputs["pixel_values"].cuda(),
            max_new_tokens=1024,
            early_stopping=False,
            do_sample=False,
            num_beams=3,
        )

    generated_text = processor.batch_decode(outputs, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=image_size
    )

    return parsed_answer, generated_text

# Function to plot image with bounding boxes
def plot_image_with_bboxes(image_np, bboxes, labels=None):
    fig, ax = plt.subplots(1)
    ax.imshow(image_np)
    colors = ['red', 'blue', 'green', 'yellow', 'purple', 'cyan']
    for i, bbox in enumerate(bboxes):
        color = colors[i % len(colors)]
        x, y, width, height = bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]
        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        if labels and i < len(labels):
            ax.text(x, y, labels[i], color=color, fontsize=8, bbox=dict(facecolor='white', alpha=0.7))
    plt.axis('off')
    return fig

# Streamlit interface
st.title("Image Processing with Florence-2-large")
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image_path = uploaded_file
    image_np, image_size = preprocess_image(image_path)

    # Image Captioning
    caption_result, _ = run_florence_model(image_np, image_size, '<CAPTION>')
    detailed_caption_result, _ = run_florence_model(image_np, image_size, '<DETAILED_CAPTION>')

    # Object Detection
    od_result, _ = run_florence_model(image_np, image_size, '<OD>')
    od_bboxes = od_result['<OD>'].get('bboxes', [])
    od_labels = od_result['<OD>'].get('labels', [])

    # OCR
    ocr_result, _ = run_florence_model(image_np, image_size, '<OCR>')

    # Phrase Grounding
    pg_result, _ = run_florence_model(image_np, image_size, '<CAPTION_TO_PHRASE_GROUNDING>', text_input=caption_result['<CAPTION>'])
    pg_bboxes = pg_result['<CAPTION_TO_PHRASE_GROUNDING>'].get('bboxes', [])
    pg_labels = pg_result['<CAPTION_TO_PHRASE_GROUNDING>'].get('labels', [])

    # Cascaded Tasks (Detailed Caption + Phrase Grounding)
    cascaded_result, _ = run_florence_model(image_np, image_size, '<CAPTION_TO_PHRASE_GROUNDING>', text_input=detailed_caption_result['<DETAILED_CAPTION>'])
    cascaded_bboxes = cascaded_result['<CAPTION_TO_PHRASE_GROUNDING>'].get('bboxes', [])
    cascaded_labels = cascaded_result['<CAPTION_TO_PHRASE_GROUNDING>'].get('labels', [])

    # Create plots
    od_fig = plot_image_with_bboxes(image_np, od_bboxes, od_labels)
    pg_fig = plot_image_with_bboxes(image_np, pg_bboxes, pg_labels)
    cascaded_fig = plot_image_with_bboxes(image_np, cascaded_bboxes, cascaded_labels)

    # Display results
    st.write(f"""
    Image Captioning:
    - Simple Caption: {caption_result['<CAPTION>']}
    - Detailed Caption: {detailed_caption_result['<DETAILED_CAPTION>']}

    Object Detection:
    - Detected {len(od_bboxes)} objects

    OCR:
    {ocr_result['<OCR>']}

    Phrase Grounding:
    - Grounded {len(pg_bboxes)} phrases from the simple caption

    Cascaded Tasks:
    - Grounded {len(cascaded_bboxes)} phrases from the detailed caption
    """)

    st.pyplot(od_fig)
    st.pyplot(pg_fig)
    st.pyplot(cascaded_fig)


Overwriting app.py


# Tunneling in order to run in colab

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.229.166.25
npx: installed 22 in 1.463s
your url is: https://thick-ravens-notice.loca.lt
